In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

### unload twoBitToFa и bedtools

In [ ]:
!wget http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa

--2021-12-01 10:11:20--  http://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64/twoBitToFa
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9367560 (8.9M)
Saving to: ‘twoBitToFa.2’

twoBitToFa.2        100%[===================>]   8.93M  12.8MB/s    in 0.7s    

2021-12-01 10:11:21 (12.8 MB/s) - ‘twoBitToFa.2’ saved [9367560/9367560]



In [ ]:
!chmod a+x twoBitToFa

In [ ]:
!apt-get install bedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bedtools is already the newest version (2.26.0+dfsg-5).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


### Подготовка Positive class

In [ ]:
data = pd.read_csv('H2AFZ_bone_marrow_intersect_with_DeepZ.bed.txt', sep='\t', comment='t', header=None)
header = ['chrom', 'chromStart', 'chromEnd']
data.columns = header[:len(data.columns)]
data.head()

,chrom,chromStart,chromEnd
0,chr1,1510394,1510398
1,chr1,3713236,3713250
2,chr1,6259857,6259930
3,chr1,6259955,6259973
4,chr1,6259985,6259999


In [ ]:
df = data[data.chromEnd - data.chromStart >= 200]
df

,chrom,chromStart,chromEnd
36,chr1,26947204,26947433
44,chr1,33592644,33593049
58,chr1,40723383,40723592
63,chr1,40974267,40974603
83,chr1,59249355,59249610
...,...,...,...
2582,chr21,27107412,27107631
2618,chr22,36877320,36877628
2638,chrX,53254403,53254621
2641,chrX,71792634,71792916


In [ ]:
df['half_len'] = (df.chromEnd - df.chromStart) / 2
df.half_len = df.half_len.astype(int)
df['chromCenter'] = df.chromStart + df.half_len
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

,chrom,chromStart,chromEnd,half_len,chromCenter
36,chr1,26947204,26947433,114,26947318
44,chr1,33592644,33593049,202,33592846
58,chr1,40723383,40723592,104,40723487
63,chr1,40974267,40974603,168,40974435
83,chr1,59249355,59249610,127,59249482
...,...,...,...,...,...
2582,chr21,27107412,27107631,109,27107521
2618,chr22,36877320,36877628,154,36877474
2638,chrX,53254403,53254621,109,53254512
2641,chrX,71792634,71792916,141,71792775


In [ ]:
df['new_start'] = df.chromCenter - 100
df['new_end'] = df.chromCenter + 100
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,chrom,chromStart,chromEnd,half_len,chromCenter,new_start,new_end
36,chr1,26947204,26947433,114,26947318,26947218,26947418
44,chr1,33592644,33593049,202,33592846,33592746,33592946
58,chr1,40723383,40723592,104,40723487,40723387,40723587
63,chr1,40974267,40974603,168,40974435,40974335,40974535
83,chr1,59249355,59249610,127,59249482,59249382,59249582
...,...,...,...,...,...,...,...
2582,chr21,27107412,27107631,109,27107521,27107421,27107621
2618,chr22,36877320,36877628,154,36877474,36877374,36877574
2638,chrX,53254403,53254621,109,53254512,53254412,53254612
2641,chrX,71792634,71792916,141,71792775,71792675,71792875


In [ ]:
for i, row in df.iterrows():
  st = row['new_start']
  end = row['new_end']
  chrom = row['chrom']
  file_name = "site_" + str(i) + ".fa"
  err = !./twoBitToFa http://hgdownload.cse.ucsc.edu/gbdb/hg19/hg19.2bit $file_name -seq=$chrom -start=$st -end=$end
print('END')

END


In [ ]:
!sed -i '1d' site_*.fa

In [ ]:
index_list = df.index.to_list()

for index in index_list:
  old_name = "site_" + str(index) + ".fa"
  new_name = "new_site_" + str(index) + ".fa"
  !tr -d "\n\r" < $old_name > $new_name

In [ ]:
!mkdir sites
!mv site_*.fa sites

mkdir: cannot create directory ‘sites’: File exists


In [ ]:
!sed -i -e '$a\' new_site_*.fa

In [ ]:
!cat new_site_*.fa > pos_class.fa

In [ ]:
!mkdir new_sites_dir
!mv new_site_*.fa new_sites_dir

mkdir: cannot create directory ‘new_sites_dir’: File exists


### Подготовка Negative class

In [ ]:
chr_dict = {"chr1" : 249250621, "chr2" : 243199373 , "chr3" : 243199373, "chr4" : 191154276, "chr5" : 180915260, 
            "chr6" : 171115067, "chr7" : 159138663, "chr8" : 146364022, "chr9" : 141213431, "chr10" : 135534747, 
            "chr11" : 135006516, "chr12" : 133851895, "chr13" : 115169878, "chr14" : 107349540, "chr15" : 102531392, 
            "chr16" : 90354753, "chr17" : 81195210, "chr18" : 78077248, "chr19" : 59128983, "chr20" : 63025520, 
            "chr21" : 48129895, "chr22" : 51304566, "chrX" : 155270560, "chrY" : 59373566}

In [ ]:
def get_non_overlaps(df):
    res = pd.DataFrame(columns=['chrom', 'chromStart', 'chromEnd'])

    for key in chr_dict:
        arr = [0]
        d = df[df.chrom == key]

        for i, row in d.iterrows():
            if row.chromStart == 0:
                arr.append(row.chromEnd)
                continue
            if row.chromEnd == chr_dict[row.chrom]:
                arr.append(row.chromStart)
                continue
            arr.append(row.chromStart)
            arr.append(row.chromEnd)
        
        if arr[-1] != chr_dict[row.chrom]:
            arr.append(chr_dict[row.chrom])
        
        for i in range(0, len(arr), 2):
            res = res.append({'chrom' : key, 'chromStart' : arr[i], 'chromEnd' : arr[i + 1]}, ignore_index=True)
    return res

In [ ]:
zdna = pd.read_csv('DeepZ.bed.txt', sep='\t', comment='t', header=None)
header = ['chrom', 'chromStart', 'chromEnd']
zdna.columns = header[:len(zdna.columns)]
zdna.head()

,chrom,chromStart,chromEnd
0,chr1,9885,10697
1,chr1,10936,11065
2,chr1,11102,11440
3,chr1,29004,29611
4,chr1,564495,565370


In [ ]:
not_zdna = get_non_overlaps(zdna)

In [ ]:
not_zdna.to_csv(r'not_zdna.bed', header=None, index=None, sep='\t', mode='a')

In [ ]:
mark = pd.read_csv('H2AFZ_bone_marrow_hg19_filtered_merged.bed.txt', sep='\t', comment='t', header=None)
header = ['chrom', 'chromStart', 'chromEnd']
mark.columns = header[:len(mark.columns)]
mark.head()

,chrom,chromStart,chromEnd
0,chr1,895532,895955
1,chr1,948517,948760
2,chr1,1092873,1093492
3,chr1,1209569,1210000
4,chr1,1406563,1406892


In [ ]:
not_mark = get_non_overlaps(mark)

In [ ]:
not_mark.to_csv(r'not_mark.bed', header=None, index=None, sep='\t', mode='a')

In [ ]:
!cat not_zdna.bed not_mark.bed > for_merge.bed

In [ ]:
!sort -k1,1 -k2,2n for_merge.bed > for_merge_sorted.bed

In [ ]:
!bedtools merge -i for_merge_sorted.bed > nots_merged.bed

In [ ]:
not_overlaps = pd.read_csv('nots_merged.bed', sep='\t', comment='t', header=None)
header = ['chrom', 'chromStart', 'chromEnd']
not_overlaps.columns = header[:len(not_overlaps.columns)]
not_overlaps.head()

,chrom,chromStart,chromEnd
0,chr1,0,1510394
1,chr1,1510398,3713236
2,chr1,3713250,6259857
3,chr1,6259930,6259955
4,chr1,6259973,6259985


In [ ]:
df = not_overlaps[not_overlaps.chromEnd - not_overlaps.chromStart >= 200]
df

,chrom,chromStart,chromEnd
0,chr1,0,1510394
1,chr1,1510398,3713236
2,chr1,3713250,6259857
5,chr1,6259999,6762104
6,chr1,6762159,8021375
...,...,...,...
2675,chrX,129306042,131352328
2676,chrX,131352364,133683328
2677,chrX,133683427,154299887
2678,chrX,154299918,155270560


In [ ]:
df['half_len'] = (df.chromEnd - df.chromStart) / 2
df.half_len = df.half_len.astype(int)
df['chromCenter'] = df.chromStart + df.half_len

df['new_start'] = df.chromCenter - 100
df['new_end'] = df.chromCenter + 100

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [ ]:
df

,chrom,chromStart,chromEnd,half_len,chromCenter,new_start,new_end
0,chr1,0,1510394,755197,755197,755097,755297
1,chr1,1510398,3713236,1101419,2611817,2611717,2611917
2,chr1,3713250,6259857,1273303,4986553,4986453,4986653
5,chr1,6259999,6762104,251052,6511051,6510951,6511151
6,chr1,6762159,8021375,629608,7391767,7391667,7391867
...,...,...,...,...,...,...,...
2675,chrX,129306042,131352328,1023143,130329185,130329085,130329285
2676,chrX,131352364,133683328,1165482,132517846,132517746,132517946
2677,chrX,133683427,154299887,10308230,143991657,143991557,143991757
2678,chrX,154299918,155270560,485321,154785239,154785139,154785339


In [ ]:
randoms = df#.sample(n=2500)

In [ ]:
randoms

,chrom,chromStart,chromEnd,half_len,chromCenter,new_start,new_end
0,chr1,0,1510394,755197,755197,755097,755297
1,chr1,1510398,3713236,1101419,2611817,2611717,2611917
2,chr1,3713250,6259857,1273303,4986553,4986453,4986653
5,chr1,6259999,6762104,251052,6511051,6510951,6511151
6,chr1,6762159,8021375,629608,7391767,7391667,7391867
...,...,...,...,...,...,...,...
2675,chrX,129306042,131352328,1023143,130329185,130329085,130329285
2676,chrX,131352364,133683328,1165482,132517846,132517746,132517946
2677,chrX,133683427,154299887,10308230,143991657,143991557,143991757
2678,chrX,154299918,155270560,485321,154785239,154785139,154785339


In [ ]:
for i, row in randoms.iterrows():
  st = row['new_start']
  end = row['new_end']
  chrom = row['chrom']
  file_name = "neg_site_" + str(i) + ".fa"
  err = !./twoBitToFa http://hgdownload.cse.ucsc.edu/gbdb/hg19/hg19.2bit $file_name -seq=$chrom -start=$st -end=$end
print('END')

END


In [ ]:
!sed -i '1d' neg_site_*.fa

In [ ]:
index_list = randoms.index.to_list()

for index in index_list:
  old_name = "neg_site_" + str(index) + ".fa"
  new_name = "new_neg_site_" + str(index) + ".fa"
  !tr -d "\n\r" < $old_name > $new_name

In [ ]:
!mkdir neg_sites
!mv neg_site_*.fa neg_sites

mkdir: cannot create directory ‘neg_sites’: File exists


In [ ]:
!sed -i -e '$a\' new_neg_site_*.fa

In [ ]:
!cat new_neg_site_*.fa > neg_class.fa

In [ ]:
!mkdir new_neg_sites_dir
!mv new_neg_site_*.fa new_neg_sites_dir

mkdir: cannot create directory ‘new_neg_sites_dir’: File exists


In [ ]:
def strip_spaces(a_str_with_spaces):
    return a_str_with_spaces['line'].replace(' ', '')

In [ ]:
neg_class = pd.read_csv('neg_class.fa', sep='\n', comment='t', header=None)
header = ['line']
neg_class.columns = header[:len(neg_class.columns)]
neg_class['line'] = neg_class.apply(strip_spaces, axis=1)
neg_class['len'] = neg_class.line.str.len() 
neg_class = neg_class[neg_class.len >= 200]

neg_class

,line,len
1,AGAACATTCCAGACCCAGAACCTTCCAATGCAGAATATTCCGGACC...,200
2,ATTTTACACTGTCCTGACAAAATATCTGAGGGCAAATACCAGTCTG...,200
6,TTTTTTACAGCAAAACCATTACAACGAAGTCCCTCCCCAAACCACC...,200
8,TATGTAATTTCAGGTAACAGACCAGTGAAAGAAAAGAGAATAGCAA...,200
9,TCATTCCCCTACTTGACCCTCTTGGGACAAAGATTCCTATAACATT...,200
...,...,...
1812,AGCAGCTCGGTGTCTCAGTTCCTCTCATTTCTGCAGCTCCTTTCAG...,200
1815,TGATCTAAGCTGAGGTGTTGTCTCAGACCCTACCGCCGAATTCTAG...,200
1818,CCGTGGCTGATGTCCATAGAACTGTAGAGTTAGAAAAGAAAATACG...,200
1821,AGGTGTTTTGAAAGGTATATCCTAAAATAGCCATCACCTTCATTTT...,200


In [ ]:
neg_class = neg_class.drop(columns='len')

In [ ]:
neg_class.to_csv(r'new_neg_class.fa', header=None, index=None, sep='\n', mode='a')

In [ ]:
pos_class = pd.read_csv('pos_class.fa', sep='\n', comment='t', header=None)
header = ['line']
pos_class.columns = header[:len(pos_class.columns)]
pos_class['line'] = pos_class.apply(strip_spaces, axis=1)
pos_class['len'] = pos_class.line.str.len() 
pos_class = pos_class[pos_class.len >= 200]

pos_class

,line,len
0,AAGGGGCGGTCCCCGGTGTCCTGCGCGGGGGCGCGGAGGGGGCGGG...,200
1,GCTCGCTTCAATATGGCTGCCCCCAGGGAGAGACGAGGCTACCATG...,200
2,GCCGCCGGACCCACCGTGTGCTCGACGCGCGAGTGAACAACACCGC...,200
4,GGCAGCCTTTTAGTGAGGAGCCGCCGCGTGGCGGGACGCAAGGAAA...,200
5,CTGCCTTCGGCCCCGGGTGGTCTCCCCCGCCCGGGGACCCCCTGTG...,200
...,...,...
267,ACCAGTTCCCGGGCCGCAGCCGCCATGGTCGCAGCGGCGGCCATTC...,200
268,gggcgcgcagcggcgcATCCTCGCGTAACCGGGCCTCTTCCCGTAA...,200
269,GGAACGCAGACCCGTGGTGAGGCCCGGGCTCCGCGTCTCCTCTGCC...,200
270,AACTCTCCGCATCCAGCCGTTCTCCCTCCCCAGTGGAAATTTGCAG...,200


In [ ]:
pos_class = pos_class.drop(columns='len')

In [ ]:
pos_class.to_csv(r'new_pos_class.fa', header=None, index=None, sep='\n', mode='a')

### Разбиваем на train/test/val

In [ ]:
pos_class = pd.read_csv('new_pos_class.fa', sep='\n', comment='t', header=None)
header = ['line']
pos_class.columns = header[:len(pos_class.columns)]
pos_class

,line
0,AAGGGGCGGTCCCCGGTGTCCTGCGCGGGGGCGCGGAGGGGGCGGG...
1,GCTCGCTTCAATATGGCTGCCCCCAGGGAGAGACGAGGCTACCATG...
2,GCCGCCGGACCCACCGTGTGCTCGACGCGCGAGTGAACAACACCGC...
3,GGCAGCCTTTTAGTGAGGAGCCGCCGCGTGGCGGGACGCAAGGAAA...
4,CTGCCTTCGGCCCCGGGTGGTCTCCCCCGCCCGGGGACCCCCTGTG...
...,...
241,ACCAGTTCCCGGGCCGCAGCCGCCATGGTCGCAGCGGCGGCCATTC...
242,gggcgcgcagcggcgcATCCTCGCGTAACCGGGCCTCTTCCCGTAA...
243,GGAACGCAGACCCGTGGTGAGGCCCGGGCTCCGCGTCTCCTCTGCC...
244,AACTCTCCGCATCCAGCCGTTCTCCCTCCCCAGTGGAAATTTGCAG...


In [ ]:
pos_test = pos_class.sample(frac=0.2)
pos_test

,line
163,AACAAACCGGATCCTTCCGCGGGGTGTCCTTCCGAGCCTCGTATAC...
172,CAGCCCGCTCTCGCTAGGACTAAGTGAGCTTCTCCCTTCTTGTTTG...
191,GGAGACTCTAGTGCGTTGGGTCGCTCATACCCCGAGGCACCCGGGG...
67,CAGGGCACAAAGGCCTACAGTCCCCACCTGAAAAAAACAGACTTGA...
100,AGGAGAGAGACTGGGTAGGCATCACCTGCCCCAGTAGCTTCCGTAC...
235,CAGAGCATGATCAGCAGCAGTCTGAGTGGAAGAGTGCCTGTGATCT...
12,CGTGTGCGTCAAACCGGGGCGACGTTGGGAGCACGCGGCGTCCCCG...
77,TAATCACGGAGTTCCAGGGAGAAGGAACTTGTGAAATGGGGGAGCC...
182,TTGTCCTCGGGCTCCGGTTACGGGCTCGTCCGCGGGACCGGCACCG...
185,ATTACAGCATGTTGCACTCCAGCGCTGCACTTCACACTCGCACACG...


In [ ]:
index_list = list(pos_test.index)

In [ ]:
pos_train = pos_class.drop(index=index_list)
pos_train

,line
0,AAGGGGCGGTCCCCGGTGTCCTGCGCGGGGGCGCGGAGGGGGCGGG...
2,GCCGCCGGACCCACCGTGTGCTCGACGCGCGAGTGAACAACACCGC...
3,GGCAGCCTTTTAGTGAGGAGCCGCCGCGTGGCGGGACGCAAGGAAA...
4,CTGCCTTCGGCCCCGGGTGGTCTCCCCCGCCCGGGGACCCCCTGTG...
5,GAGACGTGCGGAAGGCCAAGGAGGCACTGGCCGAGGTGAGCCCCGT...
...,...
239,CGAAGGCAAGAGGCCTCGAACCCTTTTGGGACCCGGAACCCATCAA...
240,CGGGGGCGGAGACCGTGGTTCCCTAAGTGGCGCAGAACTCCCGGGA...
241,ACCAGTTCCCGGGCCGCAGCCGCCATGGTCGCAGCGGCGGCCATTC...
243,GGAACGCAGACCCGTGGTGAGGCCCGGGCTCCGCGTCTCCTCTGCC...


In [ ]:
pos_val = pos_train.sample(frac=0.2)

index_list = list(pos_val.index)

pos_train = pos_train.drop(index=index_list)
pos_train

,line
0,AAGGGGCGGTCCCCGGTGTCCTGCGCGGGGGCGCGGAGGGGGCGGG...
2,GCCGCCGGACCCACCGTGTGCTCGACGCGCGAGTGAACAACACCGC...
4,CTGCCTTCGGCCCCGGGTGGTCTCCCCCGCCCGGGGACCCCCTGTG...
5,GAGACGTGCGGAAGGCCAAGGAGGCACTGGCCGAGGTGAGCCCCGT...
6,GAGGACATTTTCCTAGCATTCCGGGTACTAAAATCATTTCTTGTCC...
...,...
237,AGCAAGGTGAACGCCGGACATCGAGACATGGAATCCTAGACCGGCC...
239,CGAAGGCAAGAGGCCTCGAACCCTTTTGGGACCCGGAACCCATCAA...
240,CGGGGGCGGAGACCGTGGTTCCCTAAGTGGCGCAGAACTCCCGGGA...
241,ACCAGTTCCCGGGCCGCAGCCGCCATGGTCGCAGCGGCGGCCATTC...


In [ ]:
pos_val

,line
161,TTTCTTCGCCTAATTTGACCCGTTCTTTTTCCCCTCCTTGAAACCT...
68,ACAGAGCGGGAAGGGAAACAGGATGGGGAGAGTCAAGAGGGGAAGG...
167,TGGCGTCGTTGTTCACGGTCGCGGGGAAGCCGCCGCCGCCTCCACA...
3,GGCAGCCTTTTAGTGAGGAGCCGCCGCGTGGCGGGACGCAAGGAAA...
201,GACCCACTTCCGGTGCGTTTTACAATGAACTCCGGGCACTCGAGTC...
71,ATTAGTTCCGCTTTCCCAGCTAGTCGAATCACTGGTGCACCTCCGG...
183,CGCCTCGCCCTCCTCAGACTGGGCCACCAACACTGTCCCTACGCTT...
45,GGAGCCCGGATGCGCGCTGGGACTCGGTGCAGTCCAGCTCCACGGT...
130,ACCCAGCAGCTGCCCGCCCTCATCCTGGAACTGCACTAGCAACCGC...
38,CTTAACTGAGACGACGCTAAACTACCAGAAGATCCTTAGTGTCCGT...


In [ ]:
pos_train.to_csv(r'train.pos', header=None, index=None, sep='\n', mode='a')
pos_test.to_csv(r'test.pos', header=None, index=None, sep='\n', mode='a')
pos_val.to_csv(r'val.pos', header=None, index=None, sep='\n', mode='a')

In [ ]:
neg_class = pd.read_csv('new_neg_class.fa', sep='\n', comment='t', header=None)
header = ['line']
neg_class.columns = header[:len(neg_class.columns)]
neg_class

,line
0,AGAACATTCCAGACCCAGAACCTTCCAATGCAGAATATTCCGGACC...
1,ATTTTACACTGTCCTGACAAAATATCTGAGGGCAAATACCAGTCTG...
2,TTTTTTACAGCAAAACCATTACAACGAAGTCCCTCCCCAAACCACC...
3,TATGTAATTTCAGGTAACAGACCAGTGAAAGAAAAGAGAATAGCAA...
4,TCATTCCCCTACTTGACCCTCTTGGGACAAAGATTCCTATAACATT...
...,...
766,AGCAGCTCGGTGTCTCAGTTCCTCTCATTTCTGCAGCTCCTTTCAG...
767,TGATCTAAGCTGAGGTGTTGTCTCAGACCCTACCGCCGAATTCTAG...
768,CCGTGGCTGATGTCCATAGAACTGTAGAGTTAGAAAAGAAAATACG...
769,AGGTGTTTTGAAAGGTATATCCTAAAATAGCCATCACCTTCATTTT...


In [ ]:
neg_test = neg_class.sample(frac=0.2)

index_list = list(neg_test.index)

neg_train = neg_class.drop(index=index_list)

neg_val = neg_train.sample(frac=0.2)

index_list = list(neg_val.index)

neg_train = neg_train.drop(index=index_list)
neg_train

,line
1,ATTTTACACTGTCCTGACAAAATATCTGAGGGCAAATACCAGTCTG...
3,TATGTAATTTCAGGTAACAGACCAGTGAAAGAAAAGAGAATAGCAA...
5,GCTCCTGCCCCCTCTCAGGTGCAGACAACAGGAGCACACCGCCCCT...
7,CGTTCAGGTCATTGAAGAAGCAATATCTCCTTGGGGCCTGAAGGAG...
8,CCCCTACTCTGCTCTGAAACCTTCAGGGCCTGTCTGCCCGGTGTGA...
...,...
765,AGGCCAAGCTTGTCACTGTGAATGAGGACTGCAATGAGAGGATCCT...
766,AGCAGCTCGGTGTCTCAGTTCCTCTCATTTCTGCAGCTCCTTTCAG...
767,TGATCTAAGCTGAGGTGTTGTCTCAGACCCTACCGCCGAATTCTAG...
768,CCGTGGCTGATGTCCATAGAACTGTAGAGTTAGAAAAGAAAATACG...


In [ ]:
neg_train.to_csv(r'train.neg', header=None, index=None, sep='\n', mode='a')
neg_test.to_csv(r'test.neg', header=None, index=None, sep='\n', mode='a')
neg_val.to_csv(r'val.neg', header=None, index=None, sep='\n', mode='a')

In [ ]:
pos_test

,line
163,AACAAACCGGATCCTTCCGCGGGGTGTCCTTCCGAGCCTCGTATAC...
172,CAGCCCGCTCTCGCTAGGACTAAGTGAGCTTCTCCCTTCTTGTTTG...
191,GGAGACTCTAGTGCGTTGGGTCGCTCATACCCCGAGGCACCCGGGG...
67,CAGGGCACAAAGGCCTACAGTCCCCACCTGAAAAAAACAGACTTGA...
100,AGGAGAGAGACTGGGTAGGCATCACCTGCCCCAGTAGCTTCCGTAC...
235,CAGAGCATGATCAGCAGCAGTCTGAGTGGAAGAGTGCCTGTGATCT...
12,CGTGTGCGTCAAACCGGGGCGACGTTGGGAGCACGCGGCGTCCCCG...
77,TAATCACGGAGTTCCAGGGAGAAGGAACTTGTGAAATGGGGGAGCC...
182,TTGTCCTCGGGCTCCGGTTACGGGCTCGTCCGCGGGACCGGCACCG...
185,ATTACAGCATGTTGCACTCCAGCGCTGCACTTCACACTCGCACACG...


In [ ]:
neg_test

,line
103,TTAATCCAACCAATTCCTGTTCCTCCACTGAAATGGGAGTTTTTGG...
429,GTACATAGAACACAGGACCTACCATGAATAGTTTGGACATAACACA...
754,CCAAGGCATGAAAAGCGCGCGCTGGACGTTGCGTCAAGCCCACCGC...
225,GCTGCAAGGTGAAATAGTTGATTAGCTTTTGCCTGTATGAAGAAGT...
530,CCCAGAGGAAAAAAAAAACTACCATGTATAACTTCTCAAAGACTAT...
...,...
399,CCATCGCCTCTGCCCGCTCTGTCTTTGCGGCTCGGTCCCCGGCCGC...
683,ATTGTATTAATAGTGCTTAAGTTATAATGCTAAGGATAGCATTCTA...
698,ATCAAAGACCCATGACTATTTTATATCTCAAGTAAAATATGGCAAT...
315,AAAATCGCACGTGCCGATCCCGTAAGAAATCCAGCCGCTCTCCCTC...
